In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels as sm
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_model import ARIMA
import statistics

In [ ]:
from jupyterthemes import jtplot
jtplot.style()

# Import data

In [ ]:
#read csv file
hourly_sentiment_series = pd.read_csv(r'C:\Users\luc57.DESKTOP-NB5DC80\Side Projects\DSJ_JN\DJ_JN\Time Series\hourly_users_sentiment_subset.csv',
                                     index_col=0,
                                     parse_dates=True, squeeze=True) #squeeze to ensure format is time-series
print(hourly_sentiment_series)

In [ ]:
# Check whether the index is in datetime
print(hourly_sentiment_series.index)

In [ ]:
# Preview the data to get an idea of the values and sample size
print(hourly_sentiment_series.head())
print()
print(hourly_sentiment_series.tail())
print()
print(hourly_sentiment_series.shape) #only 24 rows of data

Time series models require data to be stationary.

In [ ]:
import matplotlib.image as mpimg
  
# Read Images
img = mpimg.imread(r'C:\Users\luc57.DESKTOP-NB5DC80\Side Projects\DSJ_JN\DJ_JN\Time Series\stationary_data.png')
  
# Output Images
plt.imshow(img)

In [ ]:
# Plot the data to check if stationary (constant mean and variance), 
# as many time series models require the data to be stationary
plt.plot(hourly_sentiment_series)
plt.show()

Data is not stationary, so we need differencing (substracting the next value by the current value).
Best not to over-difference the data because doing so can lead to inaccurate estimates.
We also want to fil the missing values to avoid problems in the modeling phase.

# Data differencing

In [ ]:
hourly_sentiment_series_diff1 = hourly_sentiment_series.diff().fillna(hourly_sentiment_series)
plt.plot(hourly_sentiment_series_diff1)
plt.show()

In [ ]:
#apply the second round of differencing to make the data look more stationary.
hourly_sentiment_series_diff2 = hourly_sentiment_series_diff1.diff().fillna(hourly_sentiment_series_diff1)
plt.plot(hourly_sentiment_series_diff2)
plt.show()

Data now looks more stationary than the first one.

# Look at ACF plot and PACF plot

to determine the number of AR terms and MA terms in ARMA model, or to spot seasonality trends.

Autoregressive --> forecast the next timestamp's value by regressing over the previous values
Moving Average --> forecasts the next timestamp's value by averaging over the previous values

Autoregressive Integrated Moving Average (ARIMA) is useful for non-stationary data and has additional seasonal differencing parameter for seasonal non-stationary data.

## ACF Plot (includes 95% confidence interval band)

Anything outside the blue band has statistically significant correlation.

In [ ]:
plot_acf(hourly_sentiment_series_diff2)
plt.show()

There is 1 major spike.

If we see a significant spike at lag x in the ACF, that helps determine the number of MA terms


## PACF Plot (95% confidence interval)

The PACF plot is a plot of the partial correlation coefficients between the series and lags of itself. In general, the "partial" correlation between two variables is the amount of correlation between them which is not explained by their mutual correlations with a specified set of other variables.

In [ ]:
plot_pacf(hourly_sentiment_series_diff2)
plt.show()

There are 2 major spikes, to determine how many AR terms.

If we see a significant spike at lag x in the PACF that helps us determine the number of AR terms.

Make sure there are no gaps between datetimes.

# Fitting the model

In [ ]:
ARMA1model_hourly_sentiment = ARIMA(hourly_sentiment_series, order=(5,2,1)).fit(transparams=False)
#5 AR terms, 2 rounds of differencing, 1 MA term
#transparams = True means things are stationary
#we set to false because we have issues with the model

print(ARMA1model_hourly_sentiment.summary())

If the p-value for a AR/MA coef is > 0.05, it's not significant enough to keep in the model.
We might want to re-model using only significant terms.
Only ma.L1.D2.users_sentiment_score is significant. 

# Make predictions

In [ ]:
# Predict the next 5 hours (5 time steps ahead), 
# which is the test/holdout set
ARMA1predict_5hourly_sentiment = ARMA1model_hourly_sentiment.predict('2/6/2019  7:00:00 PM','2/6/2019  11:00:00 PM', typ='levels')
print('Forecast/preditions for 5 hours ahead ', ARMA1predict_5hourly_sentiment)

We need to transform back the de-differenced predicted values into actual values.
In ARIMA, this is done by specify type = 'levels'.
However, we can also do it manually using cumsum.

In [ ]:
#diff2 back to diff1
undiff1 = hourly_sentiment_series_diff2.cumsum().fillna(hourly_sentiment_series_diff2)
#undiff1 back to original data
undiff2 = undiff1.cumsum().fillna(undiff1)

print(all(round(hourly_sentiment_series,6)==round(undiff2,6))) # Note: very small differences
#round 6 places past decimal points
print()
print('Original values', hourly_sentiment_series.head())
print()
print('De-differenced values', undiff2.head())

Values look the same.

In [ ]:
hourly_sentiment_series

# Plot Actual vs Predicted

We compare all values with the last 5 being actual values with all values with last 5 being predicted values.

In [ ]:
hourly_sentiment_full_actual = pd.read_csv(r'C:\Users\luc57.DESKTOP-NB5DC80\Side Projects\DSJ_JN\DJ_JN\Time Series\hourly_users_sentiment_sample.csv',
                                           index_col=0, 
                                           parse_dates=True,
                                           squeeze=True)

print('Hourly Sentiment Series: ',hourly_sentiment_series)
print()
print('Only last 5 actual values: ', hourly_sentiment_full_actual.tail())

In [ ]:
indx_row_values = hourly_sentiment_full_actual.index[19:24] 
print(indx_row_values)

In [ ]:
#we can only read series values this way
predicted_series_values = pd.Series(ARMA1predict_5hourly_sentiment, 
                                    index=['2019-02-06 19:00:00',
                                           '2019-02-06 20:00:00',
                                           '2019-02-06 21:00:00',
                                           '2019-02-06 22:00:00',
                                           '2019-02-06 23:00:00'])
print('Predicted Series Values: ', predicted_series_values)
print()
hourly_sentiment_full_predicted = hourly_sentiment_series.append(predicted_series_values)
print('Only the last five predictions: ', hourly_sentiment_full_predicted.tail())

In [ ]:
# Now let's plot actual vs predicted
plt.plot(hourly_sentiment_full_predicted, c='orange', label='predicted')
plt.plot(hourly_sentiment_full_actual, c='blue', label='actual')
plt.legend(loc='upper left')
plt.show()

In [ ]:
# Calculate the MAE to evaluate the model and see if there's 
# a big difference between actual and predicted values
actual_values_holdout = hourly_sentiment_full_actual.iloc[19:24]
predicted_values_holdout = hourly_sentiment_full_predicted.iloc[19:24]
prediction_errors = []
for i in range(len(actual_values_holdout)):
    err = actual_values_holdout[i]-predicted_values_holdout[i]
    prediction_errors.append(err)

print('Prediction errors ', prediction_errors)
mean_absolute_error = statistics.mean(map(abs, prediction_errors))
print('Mean absolute error ', mean_absolute_error)

You could also look at RMSE.

Data might not be stationary - even though looked fairly stationary to our judgement, a test would 
help better determine this

In [ ]:
# Test (using Dickey-Fuller test) to check if 2 rounds of differencing resulted in stationary data or not
test_results = adfuller(hourly_sentiment_series_diff2)

print('p-value ', test_results[1])

If > 0.05 accept the null hypothesis, as the data is non-stationary
If <= 0.05 reject the null hypothesis, as the data is stationary

What can be done next? <br>
Need to better transform these data: <br>
-Stabilize the variance by applying the cube root for neg and pos values and then difference the data <br> 
-Compare models with different AR and MA terms <br>
-This is a very small sample size of 24 timestamps, so might not have enough to spare for a holdout set  <br>
 To get more use out of your data for training, rolling over time series or timestamps at a time for different holdout sets allows for training on more timestamps; doesn't stop the model from capturing the last chunk of timestamps stored in a single holdout set <br>
-The data only looks at 24 hours in one day <br>
 Would we start to capture more of a trend in hourly sentiment if we collected data over several days?
 How would you go about collecting more data? <br>
-Look at model diagnostics <br>
-Using AIC to search best model parameters <br> 
-Handle any datetime data issues <br>
-Try other modeling techniques <br>